In [29]:
import itertools as it
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, PULP_CBC_CMD, value

# Set the possible values of each bit: {0, 1}
a_range = b_range = x_range = y_range = range(2)


def generate_bitstrings(n: int) -> list:
    """
    Function that generates all possible bitstrings of length n.
    """
    if n == 0:
        return [[]]
    else:
        previous_bitstrings = generate_bitstrings(n - 1)
        current_bitstrings = []
        for bitstring in previous_bitstrings:
            current_bitstrings.append(bitstring + [0])
            current_bitstrings.append(bitstring + [1])
        return current_bitstrings

Here, the bitstring lengths that Alice and Bob receive can be set. $x=x\_range^m$ is for Alice, $y=y\_range^n$ is for Bob.

In [30]:
# These variables may be changed
m = 1
n = 1

# These variables shouldn't be changed
# Generates all possible bitstrings for x and y
x_values = generate_bitstrings(m)
y_values = generate_bitstrings(n)

$q(x,y)$ is the distribution function, which returns the probability of Alice receiving $x$ and Bob receiving $y$. For a uniform distribution, `q_uniform` can be called.

In [31]:
def q_uniform():
    x_possibilities = len(x_values)
    y_possibilities = len(y_values)

    return 1 / (x_possibilities * y_possibilities)


# This function may be changed
def q(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    :return: A user-specified probability
    """
    return q_uniform()

Alice and Bob win the game when $a \oplus b = f(x,y)$.
$f(x,y)$ can be set here.

In [32]:
# This function may be changed
def f(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    """
    return x[0] * y[0]


# This function shouldn't be changed
def V(a: int, b: int, x: list, y: list):
    return a ^ b == f(x, y)

Here we start defining the linear program. We start by creating the variables that need to be optimized. After that we add the constraints.


In [48]:
problem = LpProblem(name='non-signalling', sense=LpMaximize)
variables = {}

# Outputs
A = [0, 1]
B = [0, 1]

expressions = []

# max sum_abxy q(x,y) V(ab|xy) p(ab|xy)
for (a, b, x, y) in it.product(A, B, x_values, y_values):
    p = LpVariable(name=f'p({a},{b}|{x},{y})', lowBound=0, upBound=1)
    variables[f'p({a},{b}|{x},{y})'] = p
    expressions.append(q(x, y) * V(a, b, x, y) * p)

problem += lpSum(expressions)


# sum_{ab} p(ab|xy) = 1  for all x,y
for (a, b) in it.product(A, B):
    constraint_sum = []
    for (x, y) in it.product(x_values, y_values):
        constraint_sum.append(variables[f'p({a},{b}|{x},{y})'])

    problem += lpSum(constraint_sum) == 1


# sum_b p(ab|xy) == sum_b p(ab|xy')  for all a,x,y,y'
for (a, x, y, y_) in it.product(A, x_values, y_values, y_values):
    if y == y_:
        continue

    # sum_b p(ab|xy)
    sum1 = []
    # sum_b p(ab|xy')
    sum2 = []

    for b in B:
        sum1.append(variables[f'p({a},{b}|{x},{y})'])
        sum2.append(variables[f'p({a},{b}|{x},{y_})'])

    problem += lpSum(sum1) == lpSum(sum2)


# sum_a p(ab|xy) == sum_a p(ab|x'y)  for all b,x,x',y
for (b, x, x_, y) in it.product(B, x_values, x_values, y_values):
    if x == x_:
        continue

    # sum_a p(ab|xy)
    sum1 = []
    # sum_a p(ab|x'y)
    sum2 = []

    for a in A:
        sum1.append(variables[f'p({a},{b}|{x},{y})'])
        sum2.append(variables[f'p({a},{b}|{x_},{y})'])

    problem += lpSum(sum1) == lpSum(sum2)



status = problem.solve()
print(LpStatus[status])

for k, v in variables.items():
    print(f'{k} = {value(v)}')

print('sum =', value(lpSum(expressions)))

Optimal
p(0,0|[0],[0]) = 1.0
p(0,0|[0],[1]) = 1.0
p(0,0|[1],[0]) = 1.0
p(0,0|[1],[1]) = 0.0
p(0,1|[0],[0]) = 0.0
p(0,1|[0],[1]) = 0.0
p(0,1|[1],[0]) = 0.0
p(0,1|[1],[1]) = 1.0
p(1,0|[0],[0]) = 0.0
p(1,0|[0],[1]) = 0.0
p(1,0|[1],[0]) = 0.0
p(1,0|[1],[1]) = 1.0
p(1,1|[0],[0]) = 1.0
p(1,1|[0],[1]) = 1.0
p(1,1|[1],[0]) = 1.0
p(1,1|[1],[1]) = 0.0
sum = 2.0
